# 신경망학습

In [1]:
import numpy as np
import sys, os

In [2]:
#손실함수
def mean_square_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

a = np.array([0, 1, 0, 1, 1])
b = np.array([0.1, 0.4, 0.1, 0.5, 0.7])

print(a - b)
print(mean_square_error(a, b))


#크로스 엔트로피 함수
#분류한 클래스에만 한정해서 실제값과 판단한 클래스값을 비교
#클래스 -> [0, 1, 0, 0, 1, 0] : One-hot 코딩
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))   # 여기서 델타는 y가 0인 경우를 대비하기 위함

[-0.1  0.6 -0.1  0.5  0.3]
0.36


In [3]:
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [6]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [7]:
def cross_entropy_error1(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y)) / batch_size